# 🛠️ Agent에서 Function 도구 사용하기

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 12px; color: white; margin-bottom: 20px;">

### ⚡ 에이전트에 기능 확장하기

사용자 정의 함수를 도구로 추가하여 에이전트의 능력을 확장하는 방법을 배웁니다.

</div>

<div style="background: #e3f2fd; border-left: 4px solid #2196f3; padding: 15px; border-radius: 8px; margin: 15px 0;">

### 📖 개요

Azure OpenAI Chat Completion 서비스를 기반으로 구축된 에이전트에서 **함수 도구(Function Tools)** 를 사용하는 방법을 살펴봅니다.

</div>

<div style="background: #fff3e0; border-left: 4px solid #ff9800; padding: 15px; border-radius: 8px; margin: 15px 0;">

### ⚠️ 중요: 에이전트 유형별 지원 여부

거의 대부분의 [에이전트 유형](SupportedAgentType.ipynb)이 **Function Calling**을 지원하지만, 일부 에이전트(Assistants 기반)는 지원하지 않습니다.   
다만, 현재 Assistants 기반 API는 2026.08에 지원종료 예정이라 사용하지 않는 것이 권장됩니다.

**지원 범위:**
| 에이전트 유형 | Function Calling | 설명 |
|--------------|-----------------|------|
| ✅ Chat Completion 기반 | 지원 | 사용자 정의 함수 제공 가능 |


💡 **이 튜토리얼에서는** Function Calling을 지원하는 Chat 클라이언트 기반 에이전트를 사용합니다.

</div>

---

## 1️⃣ Function Tool 정의하기

<div style="background: #e8f5e9; border-left: 4px solid #4caf50; padding: 15px; border-radius: 8px; margin: 15px 0;">

### 🔧 Function Tool이란?

에이전트가 필요할 때 호출할 수 있는 **사용자 정의 코드**입니다.

**만드는 방법:**
- 📝 일반 Python 함수 작성
- 🔗 에이전트 생성 시 `tools` 매개변수에 전달

</div>

### 📋 타입 어노테이션으로 설명 추가

에이전트가 함수를 더 정확하게 선택할 수 있도록 설명을 제공하는 것이 중요합니다:

**사용 도구:**
- 🐍 Python의 `Annotated` 타입 어노테이션
- 📦 Pydantic의 `Field`

**예제: 날씨 조회 함수**

다음은 특정 위치의 날씨 정보를 가져오는 간단한 함수 도구입니다:
- 함수 자체에 대한 설명 (docstring)
- 매개변수에 대한 상세 설명 (description)

In [ ]:
from typing import Annotated
from pydantic import Field

def get_weather(
    location: Annotated[str, Field(description="날씨 정보를 가져올 위치입니다. 📍")],
) -> str:
    """ 주어진 장소에 대한 날씨를 가져옵니다 """
    return f"{location}은 현재 구름이 많으며, 온도는 15°C 입니다."

### 🎨 데코레이터로 명시적 지정

`@tool` 데코레이터를 사용하여 함수의 이름과 설명을 명시적으로 지정할 수 있습니다.

**장점:**
- ✅ 함수 이름과 별도로 도구 이름 지정 가능
- ✅ 더 명확한 설명 제공
- ✅ 에이전트의 도구 선택 정확도 향상

In [ ]:
from typing import Annotated
from pydantic import Field
from agent_framework import tool

@tool(name="weather_tool", description="모든 위치의 날씨 정보를 가져옵니다 ☀️☁️")
def get_weather(
    location: Annotated[str, Field(description="날씨 정보를 가져올 위치입니다. 📍")],
) -> str:
    return f"{location}은 현재 구름이 많으며, 온도는 20°C 입니다."

<div style="background: #f5f5f5; border-left: 4px solid #9e9e9e; padding: 15px; border-radius: 8px; margin: 15px 0;">

### 💡 참고: 기본값 자동 사용

`@tool` 데코레이터에서 `name` 및 `description` 매개변수를 지정하지 않으면:

- 📌 **name**: 함수 이름 자동 사용
- 📝 **description**: 함수의 docstring 자동 사용

</div>

---

## 2️⃣ Function 도구를 사용하는 Agent 생성

<div style="background: #f3e5f5; border-left: 4px solid #9c27b0; padding: 15px; border-radius: 8px; margin: 15px 0;">

### 🚀 도구가 있는 에이전트 만들기

에이전트 생성 시 `tools` 매개변수에 함수를 전달하면 됩니다.

</div>

In [ ]:
import asyncio
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="당신은 도움이 되는 어시스턴트입니다 🤖✨",
    tools=get_weather
)

### ▶️ 에이전트 실행

이제 에이전트를 평소처럼 실행하기만 하면 됩니다. 에이전트가 필요할 때 자동으로 `get_weather` 함수 도구를 호출합니다!

**동작 흐름:**
1. 💬 사용자가 날씨 관련 질문
2. 🤖 에이전트가 질문 분석
3. 🔧 필요 시 `get_weather` 도구 호출
4. 📊 결과를 자연어로 정리하여 응답

In [ ]:
async def main():
    result = await agent.run("암스테르담의 날씨는 어떤가요? 🌧️🗺️")
    print(result.text)

await main()

---

## 3️⃣ 클래스 기반 도구: 상태 공유

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 20px; border-radius: 12px; color: white; margin: 15px 0;">

### 🎯 고급 패턴: 도구 클래스

여러 기능 도구를 메서드로 포함하는 클래스를 만들 수 있습니다!

</div>

<div style="background: #e8f5e9; border-left: 4px solid #4caf50; padding: 15px; border-radius: 8px; margin: 15px 0;">

**사용 시나리오:**
- 🗂️ **관련 기능 그룹화**: 날씨 관련 도구들을 한 클래스에
- 🔄 **상태 공유**: 도구들 간에 데이터 전달
- 📦 **코드 조직화**: 유지보수가 쉬운 구조

**예제: WeatherTools 클래스**
- `get_weather()`: 날씨 조회 + 위치 저장
- `get_weather_details()`: 마지막 위치의 상세 정보 조회

💡 **핵심**: `self.last_location`으로 상태를 공유합니다!

</div>

In [ ]:
class WeatherTools:
    def __init__(self):
        self.last_location = None

    def get_weather(
        self,
        location: Annotated[str, Field(description="날씨 정보를 가져올 위치입니다. 📍")],
    ) -> str:
        """주어진 위치의 날씨를 가져옵니다. ☀️"""
        self.last_location = location
        return f"{location}은 현재 구름이 많으며, 온도는 15°C 입니다."

    def get_weather_details(self) -> int:
        """마지막으로 요청한 위치의 상세 날씨 정보를 가져옵니다. 🌡️📊"""
        if self.last_location is None:
            return "아직 위치가 지정되지 않았습니다. ⚠️"
        return f"{self.last_location}은 현재 구름이 많으며, 최고 기온은 15°C, 최저 기온은 7°C, 습도는 60% 입니다."

### 🔗 클래스 기반 도구를 에이전트에 연결

<div style="background: #e3f2fd; border-left: 4px solid #2196f3; padding: 15px; border-radius: 8px; margin: 15px 0;">

**단계:**
1. 📦 클래스 인스턴스 생성
2. 🔧 메서드들을 리스트로 `tools`에 전달
3. 🚀 에이전트가 필요시 메서드 호출

**중요:** 인스턴스의 메서드를 전달하므로 상태가 유지됩니다!

</div>

In [ ]:
tools = WeatherTools()
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="당신은 도움이 되는 어시스턴트입니다 🤖💬",
    tools=[tools.get_weather, tools.get_weather_details]
)

---

## 4️⃣ 실행 및 결과 확인

<div style="background: #fff3e0; border-left: 4px solid #ff9800; padding: 15px; border-radius: 8px; margin: 15px 0;">

### 📊 Function 도구 호출 흐름

에이전트가 Function 도구를 호출할 때마다 해당 도구의 출력을 자동으로 받아 처리합니다.

**실행 예제:**
1. 🌆 "서울의 날씨는?" → `get_weather()` 호출 + 위치 저장
2. 🌡️ "날씨 상세 정보는?" → `get_weather_details()` 호출 (저장된 위치 사용)

**핵심:** 두 번째 질문에서 "서울"을 다시 언급하지 않아도 됩니다!

</div>

In [ ]:
# 새로운 쓰레드 생성
thread = agent.get_new_thread()

result = await agent.run("서울의 날씨는 어떤가요? 🌆☀️",thread=thread)
print("결과 1:" + result.text)

result2 = await agent.run("날씨 상세 정보는 어떻게 되나요? 🌡️📊",thread=thread)
print("결과 2:" + result2.text)